# Retrieve Stock Data and Save to DF

In [33]:
from pandas_datareader import data as web
import os
import pandas as pd
import yfinance as yf

def get_stock(ticker, start_date, end_date, s_window, l_window):
    try:
        #yf.pdr_override()
        df = yf.download(ticker, start=start_date, end=end_date,auto_adjust=False)
        print("DF: ",df)
# can use this as well        df = web.get_data_yahoo(ticker, start=start_date, end=end_date)
        df['Return'] = df['Adj Close'].pct_change()
        df['Return'].fillna(0, inplace = True)
        df['Date'] = df.index
        df['Date'] = pd.to_datetime(df['Date'])
        df['Month'] = df['Date'].dt.month
        df['Year'] = df['Date'].dt.year 
        df['Day'] = df['Date'].dt.day
        for col in ['Open', 'High', 'Low', 'Close', 'Adj Close']:
            df[col] = df[col].round(2)
        df['Weekday'] = df['Date'].dt.day_name()
        df['Week_Number'] = df['Date'].dt.strftime('%U')
        df['Year_Week'] = df['Date'].dt.strftime('%Y-%U')
        df['Short_MA'] = df['Adj Close'].rolling(window=s_window, min_periods=1).mean()
        df['Long_MA'] = df['Adj Close'].rolling(window=l_window, min_periods=1).mean()        
        col_list = ['Date', 'Year', 'Month', 'Day', 'Weekday', 
                    'Week_Number', 'Year_Week', 'Open', 
                    'High', 'Low', 'Close', 'Volume', 'Adj Close',
                    'Return', 'Short_MA', 'Long_MA']
        num_lines = len(df)
        df = df[col_list]
        print('read ', num_lines, ' lines of data for ticker: ' , ticker)
        return df
    except Exception as error:
        print(error)
        return None

In [35]:
try:
    ticker='TSLA'
    input_dir = os.getcwd()
    output_file = os.path.join(input_dir, ticker + '.csv')
    df = get_stock(ticker, start_date='2017-01-01', end_date='2025-07-08', 
               s_window=14, l_window=50)
    df.to_csv(output_file, index=False)
    print('wrote ' + str(len(df)) + ' lines to file: ' + output_file)
except Exception as e:
    print(e)
    print('failed to get Yahoo stock data for ticker: ', ticker)

[*********************100%***********************]  1 of 1 completed

DF:  Price        Adj Close       Close        High         Low        Open  \
Ticker            TSLA        TSLA        TSLA        TSLA        TSLA   
Date                                                                     
2017-01-03   14.466000   14.466000   14.688667   14.064000   14.324000   
2017-01-04   15.132667   15.132667   15.200000   14.287333   14.316667   
2017-01-05   15.116667   15.116667   15.165333   14.796667   15.094667   
2017-01-06   15.267333   15.267333   15.354000   15.030000   15.128667   
2017-01-09   15.418667   15.418667   15.461333   15.200000   15.264667   
...                ...         ...         ...         ...         ...   
2025-06-30  317.660004  317.660004  325.579987  316.600006  319.899994   
2025-07-01  300.709991  300.709991  305.890015  293.209991  298.459991   
2025-07-02  315.649994  315.649994  316.829987  303.820007  312.630005   
2025-07-03  315.350006  315.350006  318.450012  312.760010  317.989990   
2025-07-07  293.940002  293.94000


C:\Users\aksha\AppData\Local\Temp\ipykernel_17344\2185397708.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Return'].fillna(0, inplace = True)


In [36]:
df

Price,Date,Year,Month,Day,Weekday,Week_Number,Year_Week,Open,High,Low,Close,Volume,Adj Close,Return,Short_MA,Long_MA
Ticker,,,,,,,,TSLA,TSLA,TSLA,TSLA,TSLA,TSLA,,,
Date,,,,,,,,,,,,,,,,
2017-01-03,2017-01-03,2017,1,3,Tuesday,01,2017-01,14.32,14.69,14.06,14.47,88849500,14.47,NaN,14.470000,14.470000
2017-01-04,2017-01-04,2017,1,4,Wednesday,01,2017-01,14.32,15.20,14.29,15.13,168202500,15.13,0.046085,14.800000,14.800000
2017-01-05,2017-01-05,2017,1,5,Thursday,01,2017-01,15.09,15.17,14.80,15.12,88675500,15.12,-0.001057,14.906667,14.906667
2017-01-06,2017-01-06,2017,1,6,Friday,01,2017-01,15.13,15.35,15.03,15.27,82918500,15.27,0.009967,14.997500,14.997500
2017-01-09,2017-01-09,2017,1,9,Monday,02,2017-02,15.26,15.46,15.20,15.42,59692500,15.42,0.009912,15.082000,15.082000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,2025-06-30,2025,6,30,Monday,26,2025-26,319.90,325.58,316.60,317.66,76695100,317.66,-0.018447,326.457143,312.822000
2025-07-01,2025-07-01,2025,7,1,Tuesday,26,2025-26,298.46,305.89,293.21,300.71,145085700,300.71,-0.053359,324.644286,314.008800


In [ ]:
#psuedocode
'''
1. create ATR column
2. create SD column
3. build LSTM columnn
4. split into train and test
5. train lstm on training data for ATR and test
6. train lstm on training data for SD and test
'''